In [91]:
import cv2
import tf2onnx
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pickle


In [92]:
#small
dataset_s = tf.keras.utils.image_dataset_from_directory("Dataset/", batch_size=512, image_size=(120,120), shuffle=False)

Found 512 files belonging to 2 classes.


In [93]:
x = np.asarray(list(dataset_s.as_numpy_iterator())[0][0])/255
y = np.asarray(tf.one_hot(list(dataset_s.as_numpy_iterator())[0][1],depth=2))

In [94]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7,shuffle=True)

In [95]:
#Small

M1_s = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(48, 6, padding='same', strides=(4, 4), input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 5, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(2, 'sigmoid')
])

In [96]:
M1_s.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [97]:
M1_s.fit(x_train, y_train, epochs=8)

Epoch 1/8
12/12 [==============================] - 7s 225ms/step - loss: 0.3740 - accuracy: 0.9218
Epoch 2/8
 6/12 [==============>...............] - ETA: 1s - loss: 0.0061 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
M1_s.evaluate(x_test, y_test)

5/5 [==============================] - 1s 98ms/step - loss: 0.0012 - accuracy: 1.0000


[0.0011675774585455656, 1.0]

In [ ]:
cam = cv2.VideoCapture(0) # registers cam object for your webcam
key = '' # initializes key variable to contain no character.
while key != 113: # checks if key equals 113 which is ascii for q
    ret, frame = cam.read() # gets image from camera and stores it in frame
    frame = cv2.resize(frame, (120, 120)) # resizes image to 240 x 240
    infer = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # internally cv2 stores images as BGR
    # this function changes it to rgb and stores it as infer
    res = M1_s.predict(np.asarray(infer).reshape(1, 120, 120, 3)) # converts cv2 mat infer to numpy array and
    # stores the result of a prediction as res
    if res[0][1] > res[0][0]: # checks if activation for class b is higher than class a
        cv2.rectangle(frame, (0, 0), (239, 239), (0, 255, 0), 5) # if activation is higher for b draws green edge on frame
    else:
        cv2.rectangle(frame, (0, 0), (239, 239), (0, 0, 255), 5) # if activation isn't higher for b draws red edge on frame
    cv2.imshow("hello", frame) # shows frame with either red or green edge.
    key = cv2.waitKey(10) # waits 10 ms for keyboard input and stores the input in key, blank if no key pressed
cam.release() # releases the webcam device
cv2.destroyAllWindows() # destroys the named window

1/1 [==============================] - 0s 80ms/step


In [ ]:
with open("dset.pickle", "wb") as f:
    pickle.dump((x,y), f)

In [ ]:
spec = (tf.TensorSpec((None, 120, 120, 3), tf.float32, name="input"),)
output_path = "M1.onnx"
model_proto, _ = tf2onnx.convert.from_keras(M1_s, input_signature=spec, opset=13, output_path=output_path)

Could not search for non-variable resources. Concrete function internal representation may have changed.


2023-10-28 15:31:09.859143: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-10-28 15:31:09.859409: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2023-10-28 15:31:10.025591: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-10-28 15:31:10.025855: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
pass1 convert failed for name: "sequential_5/conv2d_15/Conv2D"
op: "Conv2D"
input: "input"
input: "sequential_5/conv2d_15/Conv2D/ReadVariableOp"
attr {
  key: "use_cudnn_on_gpu"
  value {
    b: true
  }
}
attr {
  key: "strides"
  value {
    list {
      i: 1
      i: 4
      i: 4
      i: 1
    }
  }
}
attr {
  key: "padding"
  value {
    s: "SAME"
  }
}
attr {
  key: "explicit_paddings"
  value {
    list {
    }
  }
}
attr {
  key: "dilations"
  value {
    list {
      i: 1
      i: 1
   

ValueError: Could not infer attribute `explicit_paddings` type from empty iterator